In [1]:
# Importando pacote request externo ao Python
import requests
from bs4 import BeautifulSoup

In [2]:
# Definindo uma função para efetuar o download da página
def download(url, num_retries=2):
    print('Downloading: ', url)
    page = None
    try:
        response = requests.get(url)
        page = response.text
        if response.status_code >= 400:
            print('Download error:', response.text)
            if num_retries and 500 <= response.status_code < 600:
                return download(url, num_retries - 1)
    except requests.exceptions.RequestExceptions as e:
        print('Download error: ', e.reason)
    return page

In [3]:
#efetuando o download do site e iniciando a 'sopa'
url = 'https://www.rottentomatoes.com/browse/tv-list-1'
html = download(url)
soup = BeautifulSoup(html, 'html.parser')

Downloading:  https://www.rottentomatoes.com/browse/tv-list-1


In [268]:
# Backup
file = open('page_q01.txt', 'w')
file.write(str(soup.prettify))
file.close()

In [293]:
# Os dados de interesse estão contidos em uma tag <script>
# Pesquiso todos os scripts da página
scripts = soup.find_all('script')
position = 0
script = ''
# Efetuo uma busca por o script que contém o nome de uma série (estudando a estrutura do site, só existe 1!)
for s in scripts:
    if s.get_text().count('The Flash') == 1:
        print('Script encontrado!' + ' Posição: %d'%(position) + ' de ' + str(len(scripts)))
        script = s.get_text()
    position+=1

Script encontrado! Posição: 38 de 54


In [278]:
# Imprimo o script
script

'\n            var mps=mps||{}; mps._queue=mps._queue||{}; mps._queue.gptloaded=mps._queue.gptloaded||[];\n            require(["globals"], function(RT) {\n                var loadPage = (function(adPromise) {\n                    require(["jquery", "underscore", "media-browser", "nouislider"], function($, _, mount, noUiSlider) {\n                        RT.browserUrlReady = $.Deferred();\n                        window.noUiSlider = noUiSlider;\n                        mount(\n                            document.getElementById(\'main-row\'), \'\', {\n                                tvListTitle1: \'New TV Tonight\',\n                                tvListTitle2: \'Most Popular TV on RT\',\n                                tvListTitle3: \'Certified Fresh TV\'\n                            }, {\n                                count: Number(\'22\'),\n                                total: Number(\'22\')\n                            },\n                            [{"title":"The Flash: Seas

In [36]:
# Blibioteca para expressões regulares
import re

In [279]:
# Pego a posição que se inicia '[{' no meu script e atribuo o valor à variável início
inicio = script.find('[{')

In [280]:
# Pego a posição que se inicia '}]' no meu script e atribuo o valor à variável fim, adicionando 2, assim pegando a posição de fim de '}]'
fim = (script.find('}]') + 2)

In [281]:
# Efetuo um slice no meu script, atribuindo esse valor à variável script
script = script[inicio:fim]

In [282]:
# Efetuo um split no meu script e adiciono o valor retornado à variável dados
dados = script.split(',')
dados

['[{"title":"The Flash: Season 4"',
 '"tomatoIcon":"fresh"',
 '"tomatoScore":100',
 '"posters":{"primary":"https://resizing.flixster.com/KBDnSHwk9VQvfZSVo4hAs9dV8ew=/800x1200/v1.dDsyNjMxMzk7ajsxNzUwODsxMjAwOzgwMDsxMjAw"}',
 '"url":"/tv/flash/s04"}',
 '{"title":"DC\'s Legends of Tomorrow: Season 3"',
 '"tomatoIcon":"fresh"',
 '"tomatoScore":100',
 '"posters":{"primary":"https://resizing.flixster.com/huYTJqqUEsXhVZjfpqunP-AFSB8=/800x1200/v1.dDsyNjMxMzg7ajsxNzUwODsxMjAwOzgwMDsxMjAw"}',
 '"url":"/tv/dc_s_legends_of_tomorrow/s03"}',
 '{"title":"At Home With Amy Sedaris: Season 1"',
 '"tomatoIcon":"fresh"',
 '"tomatoScore":100',
 '"posters":{"primary":"https://resizing.flixster.com/mWrCh15w-ayelcbQocTOA6tTJ8c=/1600x2400/v1.dDsyNjUyMzk7ajsxNzUwODsxMjAwOzE2MDA7MjQwMA"}',
 '"url":"/tv/at_home_with_amy_sedaris/s01"}',
 '{"title":"This Is Us: Season 2"',
 '"tomatoIcon":"fresh"',
 '"tomatoScore":94',
 '"posters":{"primary":"https://resizing.flixster.com/9TBkvtd1HrQrmXSqcOGnAePusZg=/1586x2379/v1.dD

In [294]:
# Efetuo uma busca por nome e avaliações, mais adiante uma função para 'limpar' os dados
nomes = []
avaliacoes = []

count = 0
while count < len(dados):
    # Se a linha conter 'title' essa linha será adicionada a minha lista nomes
    if(dados[count].count('title') == 1):
        nomes.append(dados[count])
    # Se a linha conter 'tomatoScore' essa linha será adicionada a minha lista avaliacoes
    elif(dados[count].count('tomatoScore') == 1):
        avaliacoes.append(dados[count])
    # Se a linha conter 'NA' essa linha será adicionada a minha lista avaliações, pois as avaliações podem ser números ou strings
    elif(dados[count].count('NA') == 1):
        avaliacoes.append(dados[count])
    count+=1


In [284]:
nomes

['[{"title":"The Flash: Season 4"',
 '{"title":"DC\'s Legends of Tomorrow: Season 3"',
 '{"title":"At Home With Amy Sedaris: Season 1"',
 '{"title":"This Is Us: Season 2"',
 '{"title":"Loudermilk: Season 1"',
 '{"title":"Marvel\'s Runaways: Season 1"',
 '{"title":"The Long Road Home: Miniseries"',
 '{"title":"Damnation: Season 1"',
 '{"title":"Hit the Road: Season 1"',
 '{"title":"Bull: Season 2"',
 '{"title":"NCIS: Season 15"',
 '{"title":"NCIS: New Orleans: Season 4"',
 '{"title":"The Mick: Season 2"',
 '{"title":"Fresh Off the Boat: Season 4"',
 '{"title":"Chicago Med: Season 3"',
 '{"title":"Teachers: Season 2"',
 '{"title":"Lethal Weapon: Season 2"',
 '{"title":"The Middle: Season 9"',
 '{"title":"Leah Remini: Scientology and the Aftermath: Season 2"',
 '{"title":"Major Crimes: Season 6"',
 '{"title":"Brooklyn Nine-Nine: Season 5"',
 '{"title":"If Loving You Is Wrong: Season 3"']

In [285]:
avaliacoes

['"tomatoScore":100',
 '"tomatoScore":100',
 '"tomatoScore":100',
 '"tomatoScore":94',
 '"tomatoScore":91',
 '"tomatoScore":79',
 '"tomatoScore":82',
 '"tomatoScore":56',
 '"tomatoScore":0',
 '"tomatoIcon":"NA"',
 '"tomatoIcon":"NA"',
 '"tomatoIcon":"NA"',
 '"tomatoIcon":"NA"',
 '"tomatoIcon":"NA"',
 '"tomatoIcon":"NA"',
 '"tomatoIcon":"NA"',
 '"tomatoIcon":"NA"',
 '"tomatoIcon":"NA"',
 '"tomatoIcon":"NA"',
 '"tomatoIcon":"NA"',
 '"tomatoIcon":"NA"',
 '"tomatoIcon":"NA"']

In [295]:
# Cria-se duas novas listas, para armazenar os dados limpos
nome_lst = []
avaliacoes_lst = []
count = 0
# Loop para auxiliar a percorrer as listas nomes e avaliacoes
while count < 22:
    # Pegando os nomes
    nome_texto = re.findall(r'\w+', nomes[count])
    # Deleto o nome 'title'
    del nome_texto[0]
    position = 0
    nome_str = ''
    # Loop para auxiliar a percorrer a lista nome_texto
    while position < len(nome_texto):
        # nome_str irá pegar elemento por elemento na lista texto e efetuar uma concatenação
        nome_str = nome_str + ' ' + nome_texto[position]
        position+=1
    # o nome_str é adicionado a lista nome_lst, porém antes de ser adicionado é efetuado um slice para nome_str começar sem ' '
    nome_lst.append(nome_str[1:])
    # Pegando as avaliações
    avaliacao_texto = re.findall(r'\w+', avaliacoes[count])
    # Deletando o nome 'tomatoAlgumaCoisa'
    del avaliacao_texto[0]
    position = 0
    avaliacao_str = ''
     # Loop para auxiliar a percorrer a lista avaliacao_texto
    while position < len(avaliacao_texto):
        # avaliacao_str irá pegar elemento por elemento na lista texto e efetuar uma concatenação
        avaliacao_str = avaliacao_str + ' ' + avaliacao_texto[position]
        position+=1
    # Se avaliação_str não conter 'NA', avaliação_str é convertindo para int e adicionado a lista avaliacoes_lst
    if avaliacao_str.count('NA') == 0:
        avaliacoes_lst.append(int(avaliacao_str))
    else:
        # Se conter 'NA', avaliação_str recebe o valor 'Sem Avaliação' e é adicionada a lista avaliacoes_lst
        avaliacoes_lst.append('Sem Avaliação')
    count+=1

In [296]:
nome_lst

['The Flash Season 4',
 'DC s Legends of Tomorrow Season 3',
 'At Home With Amy Sedaris Season 1',
 'This Is Us Season 2',
 'Loudermilk Season 1',
 'Marvel s Runaways Season 1',
 'The Long Road Home Miniseries',
 'Damnation Season 1',
 'Hit the Road Season 1',
 'Bull Season 2',
 'NCIS Season 15',
 'NCIS New Orleans Season 4',
 'The Mick Season 2',
 'Fresh Off the Boat Season 4',
 'Chicago Med Season 3',
 'Teachers Season 2',
 'Lethal Weapon Season 2',
 'The Middle Season 9',
 'Leah Remini Scientology and the Aftermath Season 2',
 'Major Crimes Season 6',
 'Brooklyn Nine Nine Season 5',
 'If Loving You Is Wrong Season 3']

In [297]:
avaliacoes_lst

[100,
 100,
 100,
 94,
 91,
 79,
 82,
 56,
 0,
 'Sem Avaliação',
 'Sem Avaliação',
 'Sem Avaliação',
 'Sem Avaliação',
 'Sem Avaliação',
 'Sem Avaliação',
 'Sem Avaliação',
 'Sem Avaliação',
 'Sem Avaliação',
 'Sem Avaliação',
 'Sem Avaliação',
 'Sem Avaliação',
 'Sem Avaliação']

In [300]:
# Crio um dicionário para nomes e avaliações
tomatos = {
    'nome' : nome_lst,
    'avaliação' : avaliacoes_lst
}

In [299]:
import pandas as pd
df = pd.DataFrame(tomatos, columns=['nome', 'avaliação'])
df.index = df['nome']
del df['nome']
df

,avaliação
nome,
The Flash Season 4,100
DC s Legends of Tomorrow Season 3,100
At Home With Amy Sedaris Season 1,100
This Is Us Season 2,94
Loudermilk Season 1,91
Marvel s Runaways Season 1,79
The Long Road Home Miniseries,82
Damnation Season 1,56
Hit the Road Season 1,0


In [291]:
df.describe()

,avaliação
count,22
unique,8
top,Sem Avaliação
freq,13


In [306]:
tomatos['avaliação']

[100,
 100,
 100,
 94,
 91,
 79,
 82,
 56,
 0,
 'Sem Avaliação',
 'Sem Avaliação',
 'Sem Avaliação',
 'Sem Avaliação',
 'Sem Avaliação',
 'Sem Avaliação',
 'Sem Avaliação',
 'Sem Avaliação',
 'Sem Avaliação',
 'Sem Avaliação',
 'Sem Avaliação',
 'Sem Avaliação',
 'Sem Avaliação']